In [ ]:
## String Matching with fuzzywuzzy

In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [2]:
## Reading data and creating working files :

master_data = pd.read_excel(r'C:\Users\rohan\Downloads\data.xlsx', header = 0)
tz_data = pd.read_csv(r'C:\Users\rohan\Downloads\tz.csv', header=0)

master_data_rev = master_data.dropna(subset=['COMPANY NAME']).reset_index(drop=True) # Remove blank/Nan from comany name column
tz_data_rev = tz_data.dropna(subset=['name']).reset_index(drop=True)  # #Remove blank/Nan from comany name column

master_data_rev['COMPANY NAME'] = master_data['COMPANY NAME'].astype(str).str.lower() # making names in lower case
tz_data_rev.name = tz_data_rev.name.astype(str).str.lower()  # making names in lower case

working_master = master_data_rev.copy()  # only created a working master for further processing so that ori.master remains.

## removing company name having only 2 or less characters
working_master = working_master[working_master['COMPANY NAME'].map(len) > 2].reset_index(drop=True)

## removing company name having only neumeric
working_master = working_master[~working_master['COMPANY NAME'].str.isnumeric()].reset_index(drop=True)

company_list = working_master['COMPANY NAME'].tolist()  ## master company list with which to match

c:\users\rohan\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(len(working_master))

962853


In [7]:
tz_data_short = tz_data_rev[10:num_co+10].reset_index(drop=True)
tz_data_short.head()

,id,name,mobile_no,about,company_image,no_of_employees,ownership,sector,email_blacklist,buy,...,default,gstin,gstin_type,company_profile,external_profile,address_id,company_id,created_by_id,display_email,last_modified_date
0,91696,sb interior solutions,NaN,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,0,0,NaN,86934,93953.0,hari.sbis@gmail.com,2021-04-20 15:19:57.650
1,91698,sampath,9849710101,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,NaN,Regular,0,0,92257.0,86936,93953.0,sbinteriorsolutions@yahoo.co.in,2021-04-20 15:27:24.461
2,92715,westernoverseas,NaN,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,0,0,NaN,87905,94721.0,wosa.inventory@gmail.com,2021-04-28 16:03:30.477
3,92257,reliable engineering,9930439778,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,27ABUPH2332M1ZA,Regular,0,0,92697.0,87484,18434.0,reliableengineering@outlook.com,2021-04-23 15:43:22.350
4,86867,setco spindles india pvt ltd,8669664991,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,27AAACU5820F1ZI,Regular,0,0,88585.0,82372,18434.0,Dhirajm@setco.com,2021-03-08 15:00:21.348


In [8]:
## Company matching

num_co = 10   ## number of companies to match

tz_data_short = tz_data_rev[10:num_co+10].reset_index(drop=True)  ## taking limited data as per number of companies to match

final_list = []   ## This the output list

for i in range(len(tz_data_short)):
    
    match_1 = []
    match_2 = []
    
    name_check = tz_data_short.name[i]
    ratios = process.extract(name_check, company_list, limit=2)  ## matching with master file-gives best 2 matches as per limit
    
#   print(ratios)

    id_1 = company_list.index(ratios[0][0])   ## finding index of best match
    id_2 = company_list.index(ratios[1][0])   ## finding index of 2nd best match

    match_1 = working_master.iloc[id_1].tolist()
    match_1_score = ratios[0][1]
    match_1.append(match_1_score)
    match_1.append(name_check)
    
    match_2 = working_master.iloc[id_2].tolist()
    match_2_score = ratios[1][1]
    match_2.append(match_2_score)
    match_2.append(name_check)
    
    final_list.append(match_1)                 ## Best match
    final_list.append(match_2)                 ## 2nd best match
    
## Creating dataframe from output list :
    
heading_list = ['S_No', 'Company_name', 'Add', 'City', 'Pin', 'State', 'Cont_no', 'Mob_no', 'Email', 'Web',
                'Details', 'M_score', 'Ori_name']

company_details = pd.DataFrame(final_list, columns=heading_list)   ## matched result

## Re-organise columns of above company_details:

comp_final = company_details[['Ori_name', 'S_No', 'Company_name', 'M_score', 'Add', 'City', 'Pin', 'State', 'Cont_no',
                              'Mob_no', 'Email', 'Web', 'Details']]


In [9]:
comp_final

,Ori_name,S_No,Company_name,M_score,Add,City,Pin,State,Cont_no,Mob_no,Email,Web,Details
0,sb interior solutions,439607,sb interior solutions,100,"25 Road No 1, Satavahana Nagar, L B Nagar,",Hyderabad,500036,Andhra Pradesh,40-32405747,9849710101,sbinteriorsolutions@yahoo.co.in,www.sbinteriorsolutions.com,Vinyl Flooring Dealers
1,sb interior solutions,160597,s b interior solutions,98,"Flat No 102, Bagheeratha Apartments, Chikoti G...",Hyderabad,500016,Telangana,NaN,9165851649,bdo@sbinteriorsolutions.net,http://www.sbinteriorsolutions.net,Architects
2,sampath,378851,sampath,100,Hyderabad - 500001,Hyderabad,NaN,NaN,NaN,9396994545,NaN,NaN,Physician
3,sampath,10869,sampath pvc pandhal,90,"Shop no 106, vilankurichi road, Peelamedu, Coi...",Coimbatore,NaN,Tamil Nadu,NaN,"9894086767 , 9843267004",NaN,NaN,Event Organiser
4,westernoverseas,129777,eastern overseas,90,"A2/204/205, Shah & Nahar Ind. Estate, S.J. Mar...",Mumbai,400013,Maharashtra,22-24941913,NaN,rasik@mumbaiaccessory.com,http://www.eoindia.com,"Scarves, Pareos, Shawls, Triangles, Stoles, Po..."
5,westernoverseas,105598,a & a,86,"6/627-D, GOVINDPURI, KALKAJI",Delhi,110019,Delhi,NaN,11-26447889,NaN,NaN,Plastic Card Printers
6,reliable engineering,480186,reliable engineering,100,"168/1, 4TH k. CROSS, MARUTHI NAGAR, KAMAKSHIPA...",Anand,388120,Gujarat,NaN,NaN,organictradingco@hotmail.com,NaN,"Casein, Dairy Products Dairy Products"
7,reliable engineering,54195,reliable engineering stores,95,"No.102,United Shopping Complex,94, Dr.Nanjappa...",Coimbatore,641018,Tamil Nadu,91-422-4220534,91-9894030534,NaN,www.fastenerhouse.in,"Toggle Clamps, Panel Board Accessories,Alumini..."
8,setco spindles india pvt ltd,534605,setco spindles india pvt.ltd.,100,"No.3C,K.K.Street,Kasthuri Nagar,Aynambakkam",Chennai,600095,Tamil Nadu,NaN,91-9962920917,pradeeps@setco.in,NaN,Manufacturing and reconditioning of HMC/CMC Sp...
9,setco spindles india pvt ltd,534605,setco spindles india pvt.ltd.,100,"No.3C,K.K.Street,Kasthuri Nagar,Aynambakkam",Chennai,600095,Tamil Nadu,NaN,91-9962920917,pradeeps@setco.in,NaN,Manufacturing and reconditioning of HMC/CMC Sp...


In [15]:
tz_data_short.name[4]

'setco spindles india pvt ltd'

In [18]:
name_check = tz_data_short.name[4]
ratios = process.extract(name_check, company_list, limit=3)  ## matching with master file-gives best 2 matches as per limit
    
print(ratios)

id_1 = company_list.index(ratios[0][0])   ## finding index of best match
id_2 = company_list.index(ratios[1][0])   ## finding index of 2nd best match
    
    


[('setco spindles india pvt.ltd.', 100), ('setco spindles india pvt.ltd.', 100), ('setco spindles india pvt. ltd.', 98)]


In [17]:
print(id_1)
print(id_2)

534509
534509


In [9]:
employees = [('Stuti', 28, 'Varanasi'),
            ('Saumya', 32, 'Delhi'),
            ('Aaditya', 25, 'Mumbai'),
            ('Saumya', 32, 'Delhi'),
            ('Saumya', 32, 'Delhi'),
            ('Saumya', 32, 'Mumbai'),
            ('Aaditya', 40, 'Dehradun'),
            ('Seema', 32, 'Delhi')
            ]
  
# Creating a DataFrame object
df = pd.DataFrame(employees, 
                  columns = ['Name', 'Age', 'City'])
  
# Print the Dataframe
df

,Name,Age,City
0,Stuti,28,Varanasi
1,Saumya,32,Delhi
2,Aaditya,25,Mumbai
3,Saumya,32,Delhi
4,Saumya,32,Delhi
5,Saumya,32,Mumbai
6,Aaditya,40,Dehradun
7,Seema,32,Delhi


In [92]:
grouped_df = df.groupby(['Name'])
yy = list(grouped_df.groups)
pp = grouped_df.get_group(yy[0]).reset_index(drop=True)
pp
#dfs = dict(tuple(df.groupby('Group')))

,Name,Age,City,Group
0,Aaditya,25,Mumbai,0
1,Aaditya,40,Dehradun,1


In [47]:
print(len(grouped_df))
print(xx)
yy

4
dict_keys(['Aaditya', 'Saumya', 'Seema', 'Stuti'])


['Aaditya', 'Saumya', 'Seema', 'Stuti']

In [36]:
grouped_df.get_group('Stuti')

,Name,Age,City,Group
0,Stuti,28,Varanasi,0


In [37]:
grouped_df.get_group('Saumya')

,Name,Age,City,Group
1,Saumya,32,Delhi,0
3,Saumya,32,Delhi,1
4,Saumya,32,Delhi,2
5,Saumya,32,Mumbai,3


In [23]:
for i, g in df.groupby(['Name']):
    print (g)
    print('===========')
    print(len(i))

      Name  Age      City  Group
2  Aaditya   25    Mumbai      0
6  Aaditya   40  Dehradun      0
7
     Name  Age    City  Group
1  Saumya   32   Delhi      0
3  Saumya   32   Delhi      1
4  Saumya   32   Delhi      2
5  Saumya   32  Mumbai      0
6
    Name  Age   City  Group
7  Seema   32  Delhi      0
5
    Name  Age      City  Group
0  Stuti   28  Varanasi      0
5


In [5]:
df.duplicated(subset = 'Name', keep=False)

0    False
1     True
2     True
3     True
4     True
5     True
6     True
7    False
dtype: bool

In [97]:
df[df.duplicated(subset = 'Name', keep=False)] 

,Name,Age,City
1,Saumya,32,Delhi
2,Aaditya,25,Mumbai
3,Saumya,32,Delhi
4,Saumya,32,Delhi
5,Saumya,32,Mumbai
6,Aaditya,40,Dehradun


In [6]:
df[~df.duplicated(subset = 'Name', keep=False)]

,Name,Age,City
0,Stuti,28,Varanasi
7,Seema,32,Delhi


In [92]:
df.duplicated(subset = 'Name', keep=False)

0    False
1     True
2     True
3     True
4     True
5     True
6     True
7    False
dtype: bool

In [89]:
df = pd.DataFrame(['a','b','c','d','a','b'])

# df[df.duplicated(keep=False)]
df.duplicated(keep=False)

0     True
1     True
2    False
3    False
4     True
5     True
dtype: bool

In [4]:
# make a list of all duplicate company names by assigning 'keep = False'

duplicate_master = working_master[working_master.duplicated(subset = 'COMPANY NAME', keep=False)].reset_index(drop=True)
print(len(duplicate_master))
duplicate_master.head()

110194


,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
0,9,kobelco construction equipment india pvt. ltd.,"H-200, Sector ? 63,",Noida,201307,Uttar Pradesh,91-120-4079930,91-9810183986,info@kobelconet.in,www.kobelconet.in,Manufacturer of hydraulic excavators. Factory ...
1,14,truworth impex pvt. ltd.,"Truworth House, 10, Dhuleshwar Garden,",Jaipur,302001,Rajasthan,91-141-2364370,91-9928033370,sales@truworth.com,www.truworth.in,"Truworth is a sole distributor of Bellinzoni, ..."
2,28,shree ram granimarmo pvt. ltd.,"208, ?WINDFALL?, Sahar Plaza Complex, Near Cha...",Mumbai,400059,Maharashtra,91-22-61879888,91-9820151243,NaN,www.srgm.in,"One of largest quarry owners, manufacturers an..."
3,30,mahadev marmo pvt. ltd.,"G-268-269, H ? 300-302, Udhyog Vihar, Sukher",Udaipur,313003,Rajasthan,91-294-2440642,91-9414160341,NaN,www.mahadevmarmo.com,"Stone Block , Stone Slabs , Stone Tiles , Ston..."
4,51,sanwa diamond tools pvt. ltd.,"Mysugar Building, 3rd Floor, J.C Road,",Bengaluru,560002,Karnataka,91-80-22234891,91-9916224477,NaN,www.sanwadiamondtools.com,Manufacturer's of diamond cutting tools for st...


In [13]:
## grouping all duplicate company names :

grouped_duplicate = duplicate_master.groupby(['COMPANY NAME'])
duplicate_name_list = list(grouped_duplicate.groups)
print(len(duplicate_name_list))

pp = grouped_duplicate.get_group(duplicate_name_list[18]).reset_index(drop=True)
pp


43969


,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
0,385734,32 pearls dental clinic,"Office No 1470,Chitli Qabar Chowk,Jamma Masjid...",Ghaziabad,NaN,NaN,NaN,9871020659,NaN,NaN,Physician
1,389672,32 pearls dental clinic,"C-703, Ground Floor, Opp. Shiksha Bharti Publi...",Delhi,NaN,NaN,NaN,11-33827834,anchalrana70@gmail.com,www.32pearlsclinic.com,Physician
2,806470,32 pearls dental clinic,No. 15 Muir Road Baba Halwai Chouraha Ashok Na...,Allahabad,211001,Uttar Pradesh,NaN,9956069006,docrahulmathur@gmail.com,http://www.thedentistallahabad.in,"Hospital, Clinic & Nursing Home"
3,808285,32 pearls dental clinic,Opposite Pandit Clinic Bartand Near Bus Stand,Dhanbad,826001,Jharkhand,326-2222021,9386329037,itssauravs@gmail.com,http://www.32pearlsdental.com,"Hospital, Clinic & Nursing Home"
4,810679,32 pearls dental clinic,Shop No 7 Vishnu Pratibha Complex Virar West ...,Thane,401302,Maharashtra,NaN,9920029445,machadosam1@gmail.com,http://32pearlsdentalclinic.webs.com/,"Hospital, Clinic & Nursing Home"
5,815022,32 pearls dental clinic,Raghunandan Building Shop 1 Old Sangavi Last B...,Pune,411027,Maharashtra,NaN,9970185704,gedam.sneha@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
6,816307,32 pearls dental clinic,Flat No 3 1st Floor Silent Retreate Naika Vadd...,Goa,403516,Goa,NaN,9975333937,ajit12.2008@rediffmail.com,NaN,"Hospital, Clinic & Nursing Home"
7,821125,32 pearls dental clinic,Ramgarh Cant,Ramgarh,NaN,Jharkhand,NaN,9931827669,manishkeshri4u@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
8,825178,32 pearls dental clinic,G. T. Road (E) Chandannagar Chandannagar,Hooghly,712136,West Bengal,-33,NaN,indranil2312@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
9,832546,32 pearls dental clinic,Shiv Puri Extension New Delhi,NaN,NaN,NaN,NaN,9910514170,dryog11@rediffmail.com,NaN,"Hospital, Clinic & Nursing Home"


In [ ]:
# what to keep in duplicate :

thres = 80

for i in range(len(duplicate_name_list)):
    current_group = grouped_duplicate.get_group(duplicate_name_list[i]).reset_index(drop=True)
    add_list = current_group['ADD'].tolist()
    
    for j in range(len(current_group)):
        best_match = process.extractOne(current_group['ADD'][j], add_list, scorer = fuzz.token_set_ratio)
        if best_match[1] >= thres:
            indx = add_list.index(best_match[0])  ## so, 'j' address matches with 'indx' address
            if current_group.count(1)[j] >= current_group.count(1)[indx]:
                current_group.drop(current_group.index[indx], inplace=True)
            else:
                current_group.drop(current_group.index[j], inplace=True)
            
            

In [175]:
current_group = grouped_duplicate.get_group(duplicate_name_list[0]).reset_index(drop=True)
#current_group[['ADD']] = current_group[['ADD']].replace('', np.nan, inplace=True)
current_group['ADD'].replace('   ', np.nan, inplace=True)
current_group[['ADD']] = current_group[['ADD']].fillna('nil')
#current_group[['ADD']] = current_group[['ADD']].fillna(current_group['ADD'][0])

#current_group.at[2,'ADD'] = current_group['ADD'][0]
#current_group['ADD'][6] = current_group['ADD'][0]
current_group

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
0,295,\nelektroweld automations india pvt. ltd.,"Plot NO-2402, Part-B, MIE Bahadurgarh \nPin Co...",NaN,NaN,NaN,"01276 - 267127, 267128","9810194463, 9312229293,",NaN,www.elektroweld.com\n,"maw machines, tig welding machines, mig weldin..."
1,603419,\nelektroweld automations india pvt. ltd.,"Plot NO-2402, Part-B, MIE Bahadurgarh",Jhajjar,124507,Haryana,NaN,9810194463,elektroweld96@gmail.com,NaN,"maw machines, tig welding machines, mig weldin..."


In [176]:
add_list = current_group['ADD'].tolist()
#* sss = current_group.copy()
drop_1 = []


for j in range(len(current_group)):
    drop_2 = []
    print('======================================', j)
    print('length of current_group', len(current_group))
#    print(sss['ADD'][j])
#    if len(current_group) > 1:
    if (len(current_group)-len(drop_1)) > 1:
#        comp_list = add_list.copy()
        drop_2.append(drop_1)
        drop_2.append(j)
        
#        comp_list.pop(j)
#        new list = [j for i, j in enumerate(my_list) if i not in [2, 3, 7]]
        comp_list = [k for m, k in enumerate(add_list) if m not in drop_2]
        best_match = process.extractOne(current_group['ADD'][j], comp_list, scorer = fuzz.token_set_ratio)
        print(best_match)
        
        if best_match[1] >= 80:
            add_list[j] = 'x'
            indx = add_list.index(best_match[0])  ## so, 'j' address matches with 'indx' address
            print('j ======= ', j)
            print('indx ++++++ ', indx)
            if current_group.count(1)[j] >= current_group.count(1)[indx]:
#                current_group.drop(current_group.index[indx], inplace=True)
                drop_1.append(indx)

            else:
#                current_group.drop(current_group.index[j], inplace=True)
                drop_1.append(j)
current_group.drop(current_group.index[drop_1], inplace=True)
#current_group_re = current_group.drop(current_group.index[drop_1]
current_group

====================================== 0
length of current_group 2
('Plot NO-2402, Part-B, MIE Bahadurgarh', 100)
j =======  0
indx ++++++  1
====================================== 1
length of current_group 2


,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
1,603419,\nelektroweld automations india pvt. ltd.,"Plot NO-2402, Part-B, MIE Bahadurgarh",Jhajjar,124507,Haryana,NaN,9810194463,elektroweld96@gmail.com,NaN,"maw machines, tig welding machines, mig weldin..."


In [91]:
thres = 80
choices = ["fuzzy fuzzy was a bear", "bear was fuzzy", "no comparison"]
aa = fuzz.ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
bb = fuzz.partial_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
cc = fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
dd = fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
ee = process.extract("fuzzy was a bear", choices)
ff = process.extract("fuzzy was a bear", choices, scorer = fuzz.token_set_ratio)
gg = process.extractOne("fuzzy was a bear", choices, scorer = fuzz.token_set_ratio)
print(aa, bb, cc, dd)
print(ee)
print(ff)
print(gg)
print(gg[0], 'and', type(gg[0]))
print(gg[1], 'and', type(gg[1]))
print(type(thres))
gg[0]

84 100 84 100
[('fuzzy fuzzy was a bear', 95), ('bear was fuzzy', 95), ('no comparison', 21)]
[('fuzzy fuzzy was a bear', 100), ('bear was fuzzy', 100), ('no comparison', 21)]
('fuzzy fuzzy was a bear', 100)
fuzzy fuzzy was a bear and <class 'str'>
100 and <class 'int'>
<class 'int'>


'fuzzy fuzzy was a bear'

In [5]:
# make a list of all unique company names by assigning 'keep = False' and inversing. We cannot use '.unique()' since
# it takes 1 name from duplicate names also.

unique_master = working_master[~working_master.duplicated(subset = 'COMPANY NAME', keep=False)].reset_index(drop=True)
print(len(unique_master))


852659


In [81]:
## find duplicate groups :

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
18,816307,32 pearls dental clinic,Flat No 3 1st Floor Silent Retreate Naika Vadd...,Goa,403516,Goa,NaN,9975333937,ajit12.2008@rediffmail.com,NaN,"Hospital, Clinic & Nursing Home"
19,855965,32 pearls dental clinic,57-A Vivekananda Road. Near girish park metro ...,Kolkata,700006,West Bengal,NaN,8981631339,pearls32dental@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
20,931062,32 pearls dental clinic,Unchapul,Haldwani,263139,Uttarakhand,NaN,9410166882,tripathiruchir@yahoo.com,NaN,"Hospital, Clinic & Nursing Home"
21,808285,32 pearls dental clinic,Opposite Pandit Clinic Bartand Near Bus Stand,Dhanbad,826001,Jharkhand,326-2222021,9386329037,itssauravs@gmail.com,http://www.32pearlsdental.com,"Hospital, Clinic & Nursing Home"
22,832546,32 pearls dental clinic,Shiv Puri Extension New Delhi,NaN,NaN,NaN,NaN,9910514170,dryog11@rediffmail.com,NaN,"Hospital, Clinic & Nursing Home"
23,815022,32 pearls dental clinic,Raghunandan Building Shop 1 Old Sangavi Last B...,Pune,411027,Maharashtra,NaN,9970185704,gedam.sneha@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
24,810679,32 pearls dental clinic,Shop No 7 Vishnu Pratibha Complex Virar West ...,Thane,401302,Maharashtra,NaN,9920029445,machadosam1@gmail.com,http://32pearlsdentalclinic.webs.com/,"Hospital, Clinic & Nursing Home"
25,834835,32 pearls dental clinic,,Ahmedabad,NaN,Gujarat,NaN,9428354324,jehalshah10@yahoo.co.in,NaN,"Hospital, Clinic & Nursing Home"
26,389672,32 pearls dental clinic,"C-703, Ground Floor, Opp. Shiksha Bharti Publi...",Delhi,NaN,NaN,NaN,11-33827834,anchalrana70@gmail.com,www.32pearlsclinic.com,Physician
27,806470,32 pearls dental clinic,No. 15 Muir Road Baba Halwai Chouraha Ashok Na...,Allahabad,211001,Uttar Pradesh,NaN,9956069006,docrahulmathur@gmail.com,http://www.thedentistallahabad.in,"Hospital, Clinic & Nursing Home"


In [82]:
bb[15:30]

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
15,414650,2nd's computers,"114, Ist Floor, Sahyog Building 58 Nehru Place",Delhi,110019,Delhi,11-41607930,9310033387,mereamit@yahoo.com,http://www.2ndscomputers.com/,Old / Used Computers & Peripherals / Buy Sell/...
16,545067,2x12 software solutions pvt. ltd.,"No.305, 3rd Floor, Amsri Plaza, Sarojinidevi R...",Secundrabad,500003,Andhra Pradesh,NaN,91-9000995417,srinivas.gudala@2x12.in,NaN,Restaurant management software
17,382916,32 dentz dental clinic,"Sharaf Mansion Building, Shop No 6, Ground Flo...",Mumbai,NaN,NaN,NaN,9820249661,32dentz@gmail.com,NaN,Physician
18,825178,32 pearls dental clinic,G. T. Road (E) Chandannagar Chandannagar,Hooghly,712136,West Bengal,-33,NaN,indranil2312@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
19,815022,32 pearls dental clinic,Raghunandan Building Shop 1 Old Sangavi Last B...,Pune,411027,Maharashtra,NaN,9970185704,gedam.sneha@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
20,832546,32 pearls dental clinic,Shiv Puri Extension New Delhi,NaN,NaN,NaN,NaN,9910514170,dryog11@rediffmail.com,NaN,"Hospital, Clinic & Nursing Home"
21,855965,32 pearls dental clinic,57-A Vivekananda Road. Near girish park metro ...,Kolkata,700006,West Bengal,NaN,8981631339,pearls32dental@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
22,821125,32 pearls dental clinic,Ramgarh Cant,Ramgarh,NaN,Jharkhand,NaN,9931827669,manishkeshri4u@gmail.com,NaN,"Hospital, Clinic & Nursing Home"
23,834835,32 pearls dental clinic,,Ahmedabad,NaN,Gujarat,NaN,9428354324,jehalshah10@yahoo.co.in,NaN,"Hospital, Clinic & Nursing Home"
24,389672,32 pearls dental clinic,"C-703, Ground Floor, Opp. Shiksha Bharti Publi...",Delhi,NaN,NaN,NaN,11-33827834,anchalrana70@gmail.com,www.32pearlsclinic.com,Physician


In [59]:
add_list = []
city_list = []
add_bb_list = []
city_bb_list = []

for i in range(len(aa)):
    
    add_m_ratio = fuzz.token_set_ratio(aa['ADD'][i], bb['ADD'][i])
    city_m_ratio = fuzz.token_set_ratio(aa['CITY'][i], bb['CITY'][i])
    add_list.append(add_m_ratio)
    city_list.append(city_m_ratio)
    add_bb_list.append(bb['ADD'][i])
    city_bb_list.append(bb['CITY'][i])
    
aa['add_ratio'] = add_list
aa['city_ratio'] = city_list
aa['ADD_bb'] = add_bb_list
aa['CITY_bb'] = city_bb_list

aa_final = aa[['S.No', 'COMPANY NAME', 'ADD', 'add_ratio', 'ADD_bb', 'CITY', 'city_ratio', 'CITY_bb', 'PIN', 'STATE']]
                              

aa_final.head() 

,S.No,COMPANY NAME,ADD,add_ratio,ADD_bb,CITY,city_ratio,CITY_bb,PIN,STATE
0,603419,\nelektroweld automations india pvt. ltd.,"Plot NO-2402, Part-B, MIE Bahadurgarh",100,"Plot NO-2402, Part-B, MIE Bahadurgarh \nPin Co...",Jhajjar,20,NaN,124507,Haryana
1,145738,\ngujarat machine tools,"GUJARAT HOUSE, OLD NO. 135, NEW NO. 279, LINGH...",53,"PANCHESHWER ESTATE, NEAR AMARDEEP FOUNDRY, DHE...",Chennai,15,Rajkot,NaN,Tamil Nadu
2,605523,potent water care private limited,"Shop No. 32, CSC-12, G-29, DDA Market, Sector - 3",82,"No. 32, C. S. C. - 12, G - 29, D. D. A. Market...",New Delhi,100,Delhi,110085,Delhi
3,709175,1 & 2 mharidadi overseas,NaN,10,"80/1, S.K. DEB ROAD, 5TH BY LANE, 1ST FLOOR,",Bangalore,25,Kolkata,NaN,Karnataka
4,414593,10 downing street,"No.10, My Home Tycoon Begumpet",28,25/9 Industrial Area Phase II 1st Main L B Sha...,Hyderabad,32,Chandigarh,500016,Andhra Pradesh


In [77]:
aa_final.head(20)

,S.No,COMPANY NAME,ADD,add_ratio,ADD_bb,CITY,city_ratio,CITY_bb,PIN,STATE
0,603419,\nelektroweld automations india pvt. ltd.,"Plot NO-2402, Part-B, MIE Bahadurgarh",100,"Plot NO-2402, Part-B, MIE Bahadurgarh \nPin Co...",Jhajjar,20,NaN,124507,Haryana
1,145738,\ngujarat machine tools,"GUJARAT HOUSE, OLD NO. 135, NEW NO. 279, LINGH...",53,"PANCHESHWER ESTATE, NEAR AMARDEEP FOUNDRY, DHE...",Chennai,15,Rajkot,NaN,Tamil Nadu
2,605523,potent water care private limited,"Shop No. 32, CSC-12, G-29, DDA Market, Sector - 3",82,"No. 32, C. S. C. - 12, G - 29, D. D. A. Market...",New Delhi,100,Delhi,110085,Delhi
3,709175,1 & 2 mharidadi overseas,NaN,10,"80/1, S.K. DEB ROAD, 5TH BY LANE, 1ST FLOOR,",Bangalore,25,Kolkata,NaN,Karnataka
4,414593,10 downing street,"No.10, My Home Tycoon Begumpet",28,25/9 Industrial Area Phase II 1st Main L B Sha...,Hyderabad,32,Chandigarh,500016,Andhra Pradesh
5,735414,100 feet boutique restaurant,No. 777/1 Hal 2nd Stage 100 Feet Road Indirana...,55,777/1 Indira Nagar 560038,Bengaluru,67,Bangalore,560038,Karnataka
6,696621,123 ply,B-37 Sector 9,25,"D-12/2, Phase-2Okhla Industrial Area",Noida,14,New Delhi,201301,Uttar Pradesh
7,830322,150 general hospital,Hamiltry Hospital Risory,0,,Jammu,33,Rajouri,NaN,Jammu & Kashmir
8,922179,151 base hospital,,0,,NaN,29,Gaya,NaN,NaN
9,919496,158 base hospital,,0,Bagdogra Bagdogra,Siliguri,33,Darjeeling,NaN,West Bengal


In [78]:
aa_final[20:41]

,S.No,COMPANY NAME,ADD,add_ratio,ADD_bb,CITY,city_ratio,CITY_bb,PIN,STATE
20,931062,32 pearls dental clinic,Unchapul,16,Shiv Puri Extension New Delhi,Haldwani,36,NaN,263139,Uttarakhand
21,808285,32 pearls dental clinic,Opposite Pandit Clinic Bartand Near Bus Stand,45,57-A Vivekananda Road. Near girish park metro ...,Dhanbad,29,Kolkata,826001,Jharkhand
22,832546,32 pearls dental clinic,Shiv Puri Extension New Delhi,20,Ramgarh Cant,NaN,20,Ramgarh,NaN,NaN
23,815022,32 pearls dental clinic,Raghunandan Building Shop 1 Old Sangavi Last B...,0,,Pune,15,Ahmedabad,411027,Maharashtra
24,810679,32 pearls dental clinic,Shop No 7 Vishnu Pratibha Complex Virar West ...,34,"C-703, Ground Floor, Opp. Shiksha Bharti Publi...",Thane,20,Delhi,401302,Maharashtra
25,834835,32 pearls dental clinic,,0,Flat No 3 1st Floor Silent Retreate Naika Vadd...,Ahmedabad,17,Goa,NaN,Gujarat
26,389672,32 pearls dental clinic,"C-703, Ground Floor, Opp. Shiksha Bharti Publi...",42,"Office No 1470,Chitli Qabar Chowk,Jamma Masjid...",Delhi,29,Ghaziabad,NaN,NaN
27,806470,32 pearls dental clinic,No. 15 Muir Road Baba Halwai Chouraha Ashok Na...,100,No. 15 Muir Road Baba Halwai Chouraha Ashok Na...,Allahabad,100,Allahabad,211001,Uttar Pradesh
28,825178,32 pearls dental clinic,G. T. Road (E) Chandannagar Chandannagar,41,Opposite Pandit Clinic Bartand Near Bus Stand,Hooghly,14,Dhanbad,712136,West Bengal
29,821125,32 pearls dental clinic,Ramgarh Cant,29,Shop No 7 Vishnu Pratibha Complex Virar West ...,Ramgarh,17,Thane,NaN,Jharkhand


In [62]:
type(master_data['ADD'][922178])

str

In [63]:
type(master_data['CITY'][922178])

float

In [76]:
type(master_data['WEB'][922178])

float

In [88]:
df = pd.DataFrame(['a','b','c','d','a','b'])

# df[df.duplicated(keep=False)]
df.duplicated(keep=False)

0     True
1     True
2    False
3    False
4     True
5     True
dtype: bool

In [10]:
df = pd.DataFrame(np.random.randint(0, 5, (5, 3)), columns=["A", "B","C"])
df.replace(1,np.nan,inplace=True)
df

,A,B,C
0,3.0,NaN,4
1,NaN,3.0,2
2,4.0,4.0,3
3,2.0,4.0,3
4,2.0,0.0,0


In [12]:
df.count(1)[1]

2

In [186]:
k = []
lis = ['a', 'b', 'c', 'd']
k = lis[1:]
k

['b', 'c', 'd']

In [181]:
lis

['a', 'b', 'd']